# **Install Modules**

In [1]:
#@title Install Modules
!pip install keras-unet-collection focal_loss 
!pip install tensorflow-addons

# **Path Definition and DataGenerator**

In [2]:
#@title Import Modules
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Necessary Packages
import os
import random
import cv2
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import gc
import numpy as np
import nibabel as nib
import time
import matplotlib.pyplot as plt
import glob
import time
import math
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from keras_unet_collection import models
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
import pickle
from focal_loss import BinaryFocalLoss
from IPython.display import clear_output
from keras_unet_collection.layer_utils import *
from keras_unet_collection.transformer_layers import patch_extract, patch_embedding, SwinTransformerBlock, patch_merging, patch_expanding
from keras import backend as K
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Num GPUs Available:  0


C:\Users\Asus\anaconda3\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [19]:
# @title Path Definition and DataGenerator
Image_Size = 512
Dataset = "DRIVE_AV" 
Train_Image_Path = "/kaggle/input/augmenteddataset/"+Dataset+"/images/"
Train_Label_Path = "/kaggle/input/augmenteddataset/"+Dataset+"/label/"

Test_Image_Path = "/kaggle/input/drive-av/test/images/"
Test_Label_Path = "/kaggle/input/drive-av/test/label/"

# Train_Image_Path = "/home/subin/avSegmentation/data/HRF/train/images/"
# Train_Label_Path = "/home/subin/avSegmentation/data/HRF/train/label/"

# Test_Image_Path = "/home/subin/avSegmentation/data/HRF/test/images/"
# Test_Label_Path = "/home/subin/avSegmentation/data/HRF/test/label/"

class MyDataGenerator(keras.utils.Sequence):
    def __init__(self, data_path, label_path, channel_type, batch_size=32, shuffle=True, augment_data=False):
        self.data_path = data_path
        self.label_path = label_path
        self.channel_type = channel_type
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.image_files = os.listdir(data_path)
        self.indexes = np.arange(len(self.image_files))
        self.on_epoch_end()
        self.beforeClahe = None
        self.augment_data = augment_data
        
        self.datagen = ImageDataGenerator(
            rotation_range=20,      # randomly rotate images by up to 20 degrees
            width_shift_range=0.2,  # randomly shift images horizontally by up to 20% of width
            height_shift_range=0.2, # randomly shift images vertically by up to 20% of height
            horizontal_flip=True,   # randomly flip images horizontally
            fill_mode='nearest'     # fill in missing pixels with the nearest value
#             preprocessing_function=lambda x: x + 0.05 * np.random.randn(*x.shape)
        )


    def __len__(self):
        return int(np.ceil(len(self.image_files) / self.batch_size))

    def __convertlabelto4channel__(self, y):
        y_converted = np.zeros((y.shape[0], y.shape[1], y.shape[2], 4))
        for i in range(y.shape[0]):
            y_converted[i, :, :, 0] = np.all(y[i, :, :] == [0, 0, 0], axis=-1)
            y_converted[i, :, :, 1] = np.all(y[i, :, :] == [255, 0, 0], axis=-1)
            y_converted[i, :, :, 2] = np.all(y[i, :, :] == [0, 255, 0], axis=-1)
            y_converted[i, :, :, 3] = np.all(y[i, :, :] == [0, 0, 255], axis=-1)
        return y_converted
    
    def convert_to_rgb(self, softmax_values):
        argmax_indices = np.argmax(softmax_values, axis=-1)
        indices = np.unravel_index(argmax_indices, softmax_values.shape[:-1])
        y = np.zeros_like(softmax_values)
        y[np.arange(y.shape[0])[:, None, None], indices[0], indices[1], argmax_indices] = 1
        # Convert categorical labels back to 3-channel RGB format
        rgb_labels = np.zeros((y.shape[0], y.shape[1], y.shape[2], 3), dtype=np.uint8)
        rgb_labels[np.all(y == [0, 1, 0, 0], axis=-1)] = [255, 0, 0]  # Red label
        rgb_labels[np.all(y == [0, 0, 1, 0], axis=-1)] = [0, 255, 0]  # Green label
        rgb_labels[np.all(y == [0, 0, 0, 1], axis=-1)] = [0, 0, 255]  # Blue label
        rgb_labels[np.all(y == [1, 0, 0, 0], axis=-1)] = [0, 0, 0]  # Background label
        return rgb_labels

    def getRawIInput(self):
        return self.beforeClahe

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(indexes)
        if(self.augment_data):
            X, y = self.apply_transforms(X, y)
        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        
        X = np.empty((len(indexes), Image_Size, Image_Size,1))
        y = np.empty((len(indexes), Image_Size, Image_Size, 3))
        self.beforeClahe = np.empty((len(indexes), Image_Size, Image_Size))

        for i, index in enumerate(indexes):
            # Load and pre-process image
            img_path = os.path.join(self.data_path, self.image_files[index])
            img = cv2.imread(img_path)
            img = cv2.resize(img, (Image_Size,Image_Size), interpolation = cv2.INTER_NEAREST)

            if(self.channel_type=="red"):
                img = img[:,:,2]
            elif(self.channel_type=="green"):
                img = img[:,:,1]
            elif(self.channel_type=="blue"):
                img = img[:,:,0]
            elif(self.channel_type=="gray"):
               img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            self.beforeClahe[i,] = img
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
            img = clahe.apply(img)
            
            img = np.reshape(img, (Image_Size, Image_Size, 1))
            mean = np.mean(img)
            std = np.std(img)
            normalized_img_data = (img - mean) / std
            X[i,] = normalized_img_data


            # Load label image
            
            label_path = os.path.join(self.label_path, self.image_files[index])
            label_img = cv2.imread(label_path)
            label_img = cv2.resize(label_img, (Image_Size,Image_Size), interpolation = cv2.INTER_NEAREST)
            y[i,] = label_img
        y = self.__convertlabelto4channel__(y)
        
        return X, y
    def apply_transforms(self, batch_x, batch_y):
        seed = np.random.randint(0, 100000)
        
        # apply the same set of transformations to both the input and output data
        x_transformed = self.datagen.flow(batch_x, batch_size=self.batch_size, shuffle=False, seed=seed).next()
        y_transformed = self.datagen.flow(batch_y, batch_size=self.batch_size, shuffle=False, seed=seed).next()
        
        return x_transformed, y_transformed
    def combine_and_split_data(self, train_images, train_labels, test_images, test_labels, split_percentage):
        """
        Combine train and test data and split them into new train and test data based on split percentage given

        Parameters:
        train_images (numpy.ndarray): Array of training images
        train_labels (numpy.ndarray): Array of training labels
        test_images (numpy.ndarray): Array of testing images
        test_labels (numpy.ndarray): Array of testing labels
        split_percentage (float): Percentage of data to be used for training, between 0 and 1

        Returns:
        Tuple of four numpy arrays: new_train_images, new_train_labels, new_test_images, new_test_labels
        """

        # Combine the train and test data into a single set
        all_images = np.concatenate((train_images, test_images), axis=0)
        all_labels = np.concatenate((train_labels, test_labels), axis=0)

        # Get the total number of examples in the data
        num_examples = all_images.shape[0]

        # Calculate the number of examples to use for training and testing
        num_train = int(num_examples * split_percentage)
        num_test = num_examples - num_train

        # Shuffle the data
        shuffle_indices = np.random.permutation(num_examples)
        all_images = all_images[shuffle_indices]
        all_labels = all_labels[shuffle_indices]

        # Split the data into train and test sets
        new_train_images = all_images[:num_train]
        new_train_labels = all_labels[:num_train]
        new_test_images = all_images[num_train:]
        new_test_labels = all_labels[num_train:]

        return new_train_images, new_train_labels, new_test_images, new_test_labels



# train_generator = MyDataGenerator(Train_Image_Path, Train_Label_Path, "red", batch_size=30, shuffle = True, augment_data = False) 
# X, y = train_generator.__getitem__(0)

In [20]:
with open("/kaggle/input/augmenteddataset/DRIVEAUG.pickle", "rb") as f:
    RawTrainX, RawTrainY = pickle.load(f)

In [21]:
# fig, axes = plt.subplots(nrows=X.shape[0], ncols=2, figsize=(10, 20))

# for i in range(X.shape[0]):
#     axes[i, 0].imshow(X[i])
#     axes[i, 1].imshow(y[i][:,:,1:4])
    
#     # set titles as labels
#     axes[i, 0].set_title(f"Image {i+1}")
#     axes[i, 1].set_title(f"Label {i+1}")
    
#     # remove ticks
#     axes[i, 0].set_xticks([])
#     axes[i, 0].set_yticks([])
#     axes[i, 1].set_xticks([])
#     axes[i, 1].set_yticks([])

# plt.tight_layout()
# plt.show()

In [22]:
# # @title Display first few images and labels
# train_generator = MyDataGenerator(Train_Image_Path, Train_Label_Path, "red", batch_size=16, shuffle = True) 
# X, y = train_generator.__getitem__(0)
# y_images = train_generator.convert_to_rgb(y)
# Raw_Input_Image = train_generator.getRawIInput()
# fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(20, 20))

# for i in range(4):
#     axs[0, i].imshow(Raw_Input_Image[i])
#     axs[0, i].set_title('Raw Image '+ str(i))

#     axs[1, i].imshow(X[i], cmap='gray')
#     axs[1, i].set_title('After Clahe '+ str(i))

#     axs[2, i].imshow(y[i])
#     axs[2, i].set_title('Label '+ str(i))


# # Adjust the spacing between subplots
# fig.subplots_adjust(wspace=0.2, hspace=0.2)

# # Show the plot
# plt.show()


In [23]:
# # @title Display first few images and labels of all channels

# for channelType in (["red", "green", "blue", "gray"]):
#     train_generator = MyDataGenerator(Train_Image_Path, Train_Label_Path, channelType, batch_size=16, shuffle = True)

#     X, y = train_generator.__getitem__(0)
#     y_images = train_generator.convert_to_rgb(y)
#     Raw_Input_Image = train_generator.getRawIInput()
#     fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(20, 20))

#     for i in range(4):
#         axs[0, i].imshow(Raw_Input_Image[i])
#         axs[0, i].set_title('Raw Image '+ str(i))

#         axs[1, i].imshow(X[i])
#         axs[1, i].set_title('After Clahe '+ str(i))

#         axs[2, i].imshow(y[i])
#         axs[2, i].set_title('Label '+ str(i))

#     fig.suptitle('Channel Type='+channelType, fontsize=8)
#     # Adjust the spacing between subplots
#     fig.subplots_adjust(wspace=0.2, hspace=0.2)

#     # Show the plot
#     plt.show()


# **Attention UNet Architecture**

In [24]:
from __future__ import absolute_import

from keras_unet_collection.layer_utils import *
from keras_unet_collection.activations import GELU, Snake
from keras_unet_collection._model_unet_2d import UNET_left, UNET_right
from keras_unet_collection._backbone_zoo import backbone_zoo, bach_norm_checker

from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model


def UNET_att_right(X, X_left, channel, att_channel, kernel_size=3, stack_num=2,
                   activation='ReLU', atten_activation='ReLU', attention='add',
                   unpool=True, batch_norm=False, name='right0'):
    
    pool_size = 2
    
    X = decode_layer(X, channel, pool_size, unpool, 
                     activation=activation, batch_norm=batch_norm, name='{}_decode'.format(name))
    
    X_left = attention_gate(X=X_left, g=X, channel=att_channel, activation=atten_activation, 
                            attention=attention, name='{}_att'.format(name))
    
    # Tensor concatenation
    H = concatenate([X, X_left], axis=-1, name='{}_concat'.format(name))
    
    # stacked linear convolutional layers after concatenation
    H = CONV_stack(H, channel, kernel_size, stack_num=stack_num, activation=activation, 
                   batch_norm=batch_norm, name='{}_conv_after_concat'.format(name))
    
    return H

def att_unet_2d_base(input_tensor, filter_num, stack_num_down=2, stack_num_up=2,
                     activation='ReLU', atten_activation='ReLU', attention='add', batch_norm=False, pool=True, unpool=True, 
                     backbone=None, weights='imagenet', freeze_backbone=True, freeze_batch_norm=True, name='attunet'):

    activation_func = eval(activation)

    depth_ = len(filter_num)
    X_skip = []

    # no backbone cases
    if backbone is None:
        X = input_tensor
        # downsampling blocks
        X = CONV_stack(X, filter_num[0], stack_num=stack_num_down, activation=activation, 
                       batch_norm=batch_norm, name='{}_down0'.format(name))
        X_skip.append(X)

        for i, f in enumerate(filter_num[1:]):
            X = UNET_left(X, f, stack_num=stack_num_down, activation=activation, pool=pool, 
                          batch_norm=batch_norm, name='{}_down{}'.format(name, i+1))        
            X_skip.append(X)

    else:
        # handling VGG16 and VGG19 separately
        if 'VGG' in backbone:
            backbone_ = backbone_zoo(backbone, weights, input_tensor, depth_, freeze_backbone, freeze_batch_norm)
            # collecting backbone feature maps
            X_skip = backbone_([input_tensor,])
            depth_encode = len(X_skip)

        # for other backbones
        else:
            backbone_ = backbone_zoo(backbone, weights, input_tensor, depth_-1, freeze_backbone, freeze_batch_norm)
            # collecting backbone feature maps
            X_skip = backbone_([input_tensor,])
            depth_encode = len(X_skip) + 1

        # extra conv2d blocks are applied
        # if downsampling levels of a backbone < user-specified downsampling levels
        if depth_encode < depth_:

            # begins at the deepest available tensor  
            X = X_skip[-1]

            # extra downsamplings
            for i in range(depth_-depth_encode):
                i_real = i + depth_encode

                X = UNET_left(X, filter_num[i_real], stack_num=stack_num_down, activation=activation, pool=pool, 
                              batch_norm=batch_norm, name='{}_down{}'.format(name, i_real+1))
                X_skip.append(X)

    # reverse indexing encoded feature maps
    X_skip = X_skip[::-1]
    # upsampling begins at the deepest available tensor
    X = X_skip[0]
    # other tensors are preserved for concatenation
    X_decode = X_skip[1:]
    depth_decode = len(X_decode)

    # reverse indexing filter numbers
    filter_num_decode = filter_num[:-1][::-1]

    for i in range(depth_decode):
        f = filter_num_decode[i]

        X = UNET_att_right(X, X_decode[i], f, att_channel=f//2, stack_num=stack_num_up,
                           activation=activation, atten_activation=atten_activation, attention=attention,
                           unpool=unpool, batch_norm=batch_norm, name='{}_up{}'.format(name, i))

    # if tensors for concatenation is not enough
    # then use upsampling without concatenation 
    if depth_decode < depth_-1:
        for i in range(depth_-depth_decode-1):
            i_real = i + depth_decode
            X = UNET_right(X, None, filter_num_decode[i_real], stack_num=stack_num_up, activation=activation, 
                       unpool=unpool, batch_norm=batch_norm, concat=False, name='{}_up{}'.format(name, i_real)) 
    return X

def att_unet_2d(input_size, filter_num, n_labels, stack_num_down=2, stack_num_up=2, activation='ReLU', 
                atten_activation='ReLU', attention='add', output_activation='Softmax', batch_norm=False, pool=True, unpool=True, 
                backbone=None, weights='imagenet', freeze_backbone=True, freeze_batch_norm=True, name='attunet'):
   
    
    # one of the ReLU, LeakyReLU, PReLU, ELU
    activation_func = eval(activation)
    
    if backbone is not None:
        bach_norm_checker(backbone, batch_norm)
    
    IN = Input(input_size)
    
    # base
    X = att_unet_2d_base(IN, filter_num, stack_num_down=stack_num_down, stack_num_up=stack_num_up,
                         activation=activation, atten_activation=atten_activation, attention=attention,
                         batch_norm=batch_norm, pool=pool, unpool=unpool, 
                         backbone=backbone, weights=weights, freeze_backbone=freeze_backbone, 
                         freeze_batch_norm=freeze_backbone, name=name)
    
    # output layer
    OUT = CONV_output(X, n_labels, kernel_size=1, activation=output_activation, name='{}_output'.format(name))
    
    # functional API model
    model = Model(inputs=[IN,], outputs=[OUT,], name='{}_model'.format(name))
    
    return model

# **Attention WNet Architecture**

In [25]:
def att_wnet_2d(input_size, filter_num, n_labels1, n_labels2, stack_num_down=2, stack_num_up=2, activation='ReLU', 
                atten_activation='ReLU', attention='add', output_activation='Softmax', batch_norm=False, pool=True, unpool=True, 
                backbone=None, weights='imagenet', freeze_backbone=True, freeze_batch_norm=True, name='attunet'):
    
    
    # one of the ReLU, LeakyReLU, PReLU, ELU
    activation_func = eval(activation)
    
    if backbone is not None:
        bach_norm_checker(backbone, batch_norm)
    
    IN = Input(input_size)
    
    # base
    X = att_unet_2d_base(IN, filter_num, stack_num_down=stack_num_down, stack_num_up=stack_num_up,
                         activation=activation, atten_activation=atten_activation, attention=attention,
                         batch_norm=batch_norm, pool=pool, unpool=unpool, 
                         backbone=backbone, weights=weights, freeze_backbone=freeze_backbone, 
                         freeze_batch_norm=freeze_backbone, name=name)
    
    # output layer
    OUT = CONV_output(X, n_labels1, kernel_size=1, activation=output_activation, name='{}_output'.format(name))
    
    concatLayer = tf.keras.layers.Concatenate()([IN, OUT])
    
    new_X = att_unet_2d_base(concatLayer, filter_num, stack_num_down=stack_num_down, stack_num_up=stack_num_up,
                         activation=activation, atten_activation=atten_activation, attention=attention,
                         batch_norm=batch_norm, pool=pool, unpool=unpool, 
                         backbone=backbone, weights=weights, freeze_backbone=freeze_backbone, 
                         freeze_batch_norm=freeze_backbone, name=name+"second")
    OUT2 = CONV_output(new_X, n_labels2,kernel_size=1, activation=output_activation, name='{}_output'.format(name+"second"))
    
    # functional API model
    model = Model(inputs=[IN,], outputs=[OUT2,], name='{}_model'.format(name))
    
    return model


# **Swin UNet Architecture**

In [26]:


def swin_transformer_stack(X, stack_num, embed_dim, num_patch, num_heads, window_size, num_mlp, shift_window=True, name=''):

    # Turn-off dropouts
    mlp_drop_rate = 0 # Droupout after each MLP layer
    attn_drop_rate = 0 # Dropout after Swin-Attention
    proj_drop_rate = 0 # Dropout at the end of each Swin-Attention block, i.e., after linear projections
    drop_path_rate = 0 # Drop-path within skip-connections
    
    qkv_bias = True # Convert embedded patches to query, key, and values with a learnable additive value
    qk_scale = None # None: Re-scale query based on embed dimensions per attention head # Float for user specified scaling factor
    
    if shift_window:
        shift_size = window_size // 2
    else:
        shift_size = 0
    
    for i in range(stack_num):
    
        if i % 2 == 0:
            shift_size_temp = 0
        else:
            shift_size_temp = shift_size

        X = SwinTransformerBlock(dim=embed_dim, num_patch=num_patch, num_heads=num_heads, 
                                 window_size=window_size, shift_size=shift_size_temp, num_mlp=num_mlp, qkv_bias=qkv_bias, qk_scale=qk_scale,
                                 mlp_drop=mlp_drop_rate, attn_drop=attn_drop_rate, proj_drop=proj_drop_rate, drop_path_prob=drop_path_rate, 
                                 name='name{}'.format(i))(X)
    return X


def swin_unet_2d_base(input_tensor, filter_num_begin, depth, stack_num_down, stack_num_up, 
                      patch_size, num_heads, window_size, num_mlp, shift_window=True, name='swin_unet'):
   
    # Compute number be patches to be embeded
    input_size = input_tensor.shape.as_list()[1:]
    num_patch_x = input_size[0]//patch_size[0]
    num_patch_y = input_size[1]//patch_size[1]
    
    # Number of Embedded dimensions
    embed_dim = filter_num_begin
    
    depth_ = depth
    
    X_skip = []

    X = input_tensor
    
    # Patch extraction
    X = patch_extract(patch_size)(X)

    # Embed patches to tokens
    X = patch_embedding(num_patch_x*num_patch_y, embed_dim)(X)
    
    # The first Swin Transformer stack
    X = swin_transformer_stack(X, stack_num=stack_num_down, 
                               embed_dim=embed_dim, num_patch=(num_patch_x, num_patch_y), 
                               num_heads=num_heads[0], window_size=window_size[0], num_mlp=num_mlp, 
                               shift_window=shift_window, name='{}_swin_down0'.format(name))
    X_skip.append(X)
    
    # Downsampling blocks
    for i in range(depth_-1):
        
        # Patch merging
        X = patch_merging((num_patch_x, num_patch_y), embed_dim=embed_dim, name='down{}'.format(i))(X)
        
        # update token shape info
        embed_dim = embed_dim*2
        num_patch_x = num_patch_x//2
        num_patch_y = num_patch_y//2
        
        # Swin Transformer stacks
        X = swin_transformer_stack(X, stack_num=stack_num_down, 
                                   embed_dim=embed_dim, num_patch=(num_patch_x, num_patch_y), 
                                   num_heads=num_heads[i+1], window_size=window_size[i+1], num_mlp=num_mlp, 
                                   shift_window=shift_window, name='{}_swin_down{}'.format(name, i+1))
        
        # Store tensors for concat
        X_skip.append(X)
        
    # reverse indexing encoded tensors and hyperparams
    X_skip = X_skip[::-1]
    num_heads = num_heads[::-1]
    window_size = window_size[::-1]
    
    # upsampling begins at the deepest available tensor
    X = X_skip[0]
    
    # other tensors are preserved for concatenation
    X_decode = X_skip[1:]
    
    depth_decode = len(X_decode)
    
    for i in range(depth_decode):
        
        # Patch expanding
        X = patch_expanding(num_patch=(num_patch_x, num_patch_y),
                            embed_dim=embed_dim, upsample_rate=2, return_vector=True, name='{}_swin_up{}'.format(name, i))(X)
        

        # update token shape info
        embed_dim = embed_dim//2
        num_patch_x = num_patch_x*2
        num_patch_y = num_patch_y*2
        
        # Concatenation and linear projection
        X = concatenate([X, X_decode[i]], axis=-1, name='{}_concat_{}'.format(name, i))
        X = Dense(embed_dim, use_bias=False, name='{}_concat_linear_proj_{}'.format(name, i))(X)
        
        # Swin Transformer stacks
        X = swin_transformer_stack(X, stack_num=stack_num_up, 
                           embed_dim=embed_dim, num_patch=(num_patch_x, num_patch_y), 
                           num_heads=num_heads[i], window_size=window_size[i], num_mlp=num_mlp, 
                           shift_window=shift_window, name='{}_swin_up{}'.format(name, i))
        
    # The last expanding layer; it produces full-size feature maps based on the patch size
    # !!! <--- "patch_size[0]" is used; it assumes patch_size = (size, size)
    X = patch_expanding(num_patch=(num_patch_x, num_patch_y),
                        embed_dim=embed_dim, upsample_rate=patch_size[0], return_vector=False)(X)
    
    return X

# def create_mask(input_tensor):
#         input_size = input_tensor.shape.as_list()[1:]
# #         all_data_tensor = tensor.empty((0, input_tensor.shape[0]), dtype=torch.float32)
        
# #         print()
#         newi = input_tensor.shape[0]
#         if(newi == None):
#             newi = 1
# #         new_label = tf.zeros((newi,512,512,3))
#         input_tensor1  = Input(shape=(newi,512,512,3))
#         new_label = K.get_value(input_tensor1)
#         for i in range(newi):
#             for j in range(input_tensor.shape[1]):
#                 for k in range(input_tensor.shape[2]):
#                     index = tf.math.argmax(input_tensor[i][j][k])
#                     tf.keras.backend.set_value(new_label[i][j][k][index-1], 1)
                    
# #                     new_label[i][j][index-1].assign(1)
#                     break
# #                     print(tf.get_static_value(input_tensor[i][j][k]))
# #                     print(input_tensor[i][j][k].print())
# #                     print(tf.math.argmax(input_tensor[i][j][k]))
#                 break
#             break
        
# #                 index = np.argmax(input_tensor[i][j])
# #                 if(index>0):
# #                     new_label[i][j][index-1]=255
#         K.set_value(input_tensor1, new_label)
#         return input_tensor1

def swin_unet_2d(input_size, filter_num_begin, n_labels, depth, stack_num_down, stack_num_up, 
                      patch_size, num_heads, window_size, num_mlp, output_activation='Softmax', shift_window=True, name='swin_unet'):
    
    IN = Input(input_size)
    
    # base    
    X = swin_unet_2d_base(IN, filter_num_begin=filter_num_begin, depth=depth, stack_num_down=stack_num_down, stack_num_up=stack_num_up, 
                          patch_size=patch_size, num_heads=num_heads, window_size=window_size, num_mlp=num_mlp, shift_window=shift_window, name=name)
    
    # output layer
    OUT = CONV_output(X, n_labels, kernel_size=1, activation=output_activation, name='{}_output'.format(name))
#   
#     NEW_OUT = create_mask(OUT)
#     concatLayer = tf.keras.layers.Concatenate()([IN, OUT])
#     new_X = swin_unet_2d_base(concatLayer, filter_num_begin=filter_num_begin, depth=depth, stack_num_down=stack_num_down, stack_num_up=stack_num_up, 
#                           patch_size=patch_size, num_heads=num_heads, window_size=window_size, num_mlp=num_mlp, shift_window=shift_window, name=name+"second")
    
    
    
    
    
#     OUT2 = CONV_output(new_X, n_labels, kernel_size=1, activation=output_activation, name='{}_output'.format(name+"second"))
    
    
    # functional API model
    model = Model(inputs=[IN,], outputs=[OUT,], name='{}_model'.format(name))
    
    return model

# model_alias = f"1_swin_unet_normal"
# model = swin_unet_2d( (512,512,3) , filter_num_begin=16, n_labels=2, 
#                                     depth=3, stack_num_down=2, stack_num_up=2, patch_size=(2, 2), 
#                                     num_heads=[4, 8, 8, 8], window_size=[4, 2, 2, 2], num_mlp=128, 
#                                 output_activation='Softmax', shift_window=True, name=model_alias)
# model.summary()

# **Swin WNet Architecture**

In [27]:
def swin_wnet_2d(input_size, filter_num_begin, n_labels1,n_labels2, depth, stack_num_down, stack_num_up, 
                      patch_size, num_heads, window_size, num_mlp, output_activation='Softmax', shift_window=True, name='swin_unet'):

    IN = Input(input_size)
    
    # base    
    X = swin_unet_2d_base(IN, filter_num_begin=filter_num_begin, depth=depth, stack_num_down=stack_num_down, stack_num_up=stack_num_up, 
                          patch_size=patch_size, num_heads=num_heads, window_size=window_size, num_mlp=num_mlp, shift_window=shift_window, name=name)
    
    # output layer
    OUT = CONV_output(X, n_labels1, kernel_size=1, activation=output_activation, name='{}_output'.format(name))
#   
    concatLayer = tf.keras.layers.Concatenate()([IN[:,:,:,0:3], OUT])
    new_X = swin_unet_2d_base(concatLayer, filter_num_begin=filter_num_begin, depth=depth, stack_num_down=stack_num_down, stack_num_up=stack_num_up, 
                          patch_size=patch_size, num_heads=num_heads, window_size=window_size, num_mlp=num_mlp, shift_window=shift_window, name=name+"second")
    
    
    
    
    
    OUT2 = CONV_output(new_X, n_labels2, kernel_size=1, activation=output_activation, name='{}_output'.format(name+"second"))
    
    
    # functional API model
    model = Model(inputs=[IN,], outputs=[OUT2,], name='{}_model'.format(name+"second"))
    
    return model

# model_alias = f"1_swin_Wnet_normal"
# model = swin_wnet_2d( (512,512,3) , filter_num_begin=16, n_labels1=2, n_labels2=4,
#                                     depth=3, stack_num_down=2, stack_num_up=2, patch_size=(2, 2), 
#                                     num_heads=[4, 8, 8, 8], window_size=[4, 2, 2, 2], num_mlp=128, 
#                                 output_activation='Softmax', shift_window=True, name=model_alias)
# model.summary()

# **Focal Loss Definition, Model Checkpoint, Optimizer and Callback definitions**

In [28]:

def focal_weighted(y_true,y_pred):
    GAMMA_FOUR_CLASS = [ 2, 2, 2, 2]
    POS_WEIGHT_FOUR_CLASS = [ 0.1, 0.8, 0.9, 0.8]

    loss = 0
    w = 2
    gamma = 2
    edge_gamma = 2
    # loss_func = BinaryFocalLoss(gamma=GAMMA_FOUR_CLASS[0],pos_weight = POS_WEIGHT_FOUR_CLASS[0])
    # loss = loss + loss_func(y_true[:,:,:,0],y_pred[:,:,:,0])

    loss_func = BinaryFocalLoss(gamma=GAMMA_FOUR_CLASS[1],pos_weight = POS_WEIGHT_FOUR_CLASS[1])
    loss = loss + loss_func(y_true[:,:,:,1],y_pred[:,:,:,1])

    loss_func = BinaryFocalLoss(gamma=GAMMA_FOUR_CLASS[2],pos_weight = POS_WEIGHT_FOUR_CLASS[2])
    loss = loss + loss_func(y_true[:,:,:,2],y_pred[:,:,:,2])

    loss_func = BinaryFocalLoss(gamma=GAMMA_FOUR_CLASS[3],pos_weight = POS_WEIGHT_FOUR_CLASS[3])
    loss = loss + loss_func(y_true[:,:,:,3],y_pred[:,:,:,3])
    
    return loss

In [29]:
#@title Definitions - DisplayCallback()
class DisplayCallback(keras.callbacks.Callback):
    def __init__(self,model, train_images,train_labels, epoch_interval=5):
        self.model = model
        self.train = train_images
        self.label = train_labels
        self.epoch_interval = epoch_interval
    
    def display(self, display_list, extra_title=''):
        plt.figure(figsize=(15, 15))
        title = ['Input Image', 'True Mask', 'Predicted Mask']

        if len(display_list) > len(title):
            title.append(extra_title)

        for i in range(len(display_list)):
            plt.subplot(1, len(display_list), i+1)
            plt.title(title[i])
            plt.imshow(display_list[i], cmap='gray')
            plt.axis('off')
        plt.show()
        
    def create_mask(self, pred_mask):
        new_label = np.zeros((Image_Size,Image_Size,3))
        for i in range(Image_Size):
            for j in range(Image_Size):
                index = np.argmax(pred_mask[i][j])
                if(index>0):
                    new_label[i][j][index-1]=255
        return new_label
    
    def show_predictions(self, train,label, num=1):
#         index = random.randint(0, 9)
        pred_mask = self.model.predict(train)
        print(type(pred_mask))
        print(pred_mask.shape)

        self.display([train[0], self.create_mask(label[0]), self.create_mask(pred_mask[0])])
#         for image, mask in dataset.take(num):
#             pred_mask = model.predict(image)
#             self.display([image[0], mask[0], self.create_mask(pred_mask)])
        
    def on_epoch_end(self, epoch, logs=None):
        if epoch and epoch % self.epoch_interval == 0:
            self.show_predictions(self.train, self.label)
            print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

In [30]:
#@title Definitions - generate_callbacks(), generate_loss(), generate_optimizer()

def generate_callbacks(model, CHECKPOINT_MODE, CHECKPOINT_PATH, CHECKPOINT_MONITOR, CALL_BACK_INTERVAL,SampleTestImage, SampleTestLabel):
    checkpoint = ModelCheckpoint(CHECKPOINT_PATH, monitor=CHECKPOINT_MONITOR, verbose=1, save_best_only=True, mode=CHECKPOINT_MODE)
    earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=200, mode="min")
    callbacks_list = [DisplayCallback(model, SampleTestImage, SampleTestLabel, CALL_BACK_INTERVAL), checkpoint, earlyStopping]
    
    return callbacks_list

def generate_loss(loss_function):
    if(loss_function == "sparse"):
        loss = "sparse_categorical_crossentropy"
    elif(loss_function == "catcross"):
        loss = "categorical_crossentropy"
    elif(loss_function == "focal_loss"):
        loss = focal_weighted
    return loss
        

def generate_optimizer(optimizer, learning_rate):
    if(optimizer == "Adam"):
        opt = tf.optimizers.Adam(learning_rate)
        return opt
    if(optimizer == "Lamb"):
        opt = tfa.optimizers.LAMB(learning_rate = learning_rate, weight_decay = 2*(1e-2) )
        return opt


# **Train and Test Data Generator**

In [31]:
import cv2
import numpy as np

def green_channel_clahe_zscore(images):
    # Initialize an empty array to store the processed images
    processed_images = []

    # Loop through the input images
    for img in images:
        # Extract the green channel
        green_channel = img[:, :, 1]

        # Apply CLAHE to enhance contrast
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        green_clahe = clahe.apply(green_channel)

        # Z-score normalization
        mean = np.mean(green_clahe)
        std = np.std(green_clahe)
        green_zscore = (green_clahe - mean) / std

        # Append the processed image to the output array
        processed_images.append(green_zscore)

    # Convert the output array to a numpy array
    processed_images = np.array(processed_images)
    processed_images = np.expand_dims(processed_images, axis=-1)

    return processed_images

In [33]:
ChannelType = "green"
# if(Dataset == "DRIVE_AV"):
#     train_generator = MyDataGenerator(Train_Image_Path, Train_Label_Path, ChannelType, batch_size=30, shuffle = False)
#     test_generator = MyDataGenerator(Test_Image_Path, Test_Label_Path, ChannelType, batch_size=30, shuffle = False)
# elif(Dataset == "HRF_AV"):
#     test_generator = MyDataGenerator(Train_Image_Path, Train_Label_Path, ChannelType, batch_size=30, shuffle = False)
#     train_generator = MyDataGenerator(Test_Image_Path, Test_Label_Path, ChannelType, batch_size=30, shuffle = False)

# TrainX, TrainY = train_generator.__getitem__(0)
test_generator = MyDataGenerator(Test_Image_Path, Test_Label_Path, ChannelType, batch_size=6, shuffle = False)
TestX, TestY = test_generator.__getitem__(0)

print(TestX.shape, TestY.shape)

(6, 512, 512, 1) (6, 512, 512, 4)


In [34]:
TrainX = green_channel_clahe_zscore(RawTrainX)
TrainY = test_generator.__convertlabelto4channel__(RawTrainY)

In [35]:
no_epoch = 200
n_labels = 4
optimizers = "Adam"
losses = "focal_loss"
batch_sizes = 6
learning_rate = 1e-2

# **Initiate and Train Model** 
### **Note:** Run either one of the cell, i.e Attention-WNet and Swin-WNet. 

In [36]:
model_alias = f"1_Attention_WNet_normal"
model = att_wnet_2d((Image_Size, Image_Size, 1), filter_num=[32, 64, 128, 256], n_labels1 =n_labels, n_labels2=n_labels,
                                stack_num_down=2, stack_num_up=2, activation='ReLU', 
                                atten_activation='ReLU', attention='add', output_activation='Softmax', 
                                batch_norm=True, pool='max', unpool=False, backbone=None, weights=None, 
                                name='attwnet')
model.summary()

Model: "attwnet_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 1  0           []                               
                                )]                                                                
                                                                                                  
 attwnet_down0_0 (Conv2D)       (None, 512, 512, 32  288         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 attwnet_down0_0_bn (BatchNorma  (None, 512, 512, 32  128        ['attwnet_down0_0[0][0]']        
 lization)                      )                                                     

In [ ]:
model_alias = f"1_swin_Wnet_normal"
#model = swin_wnet_2d( (Image_Size,Image_Size,1) , filter_num_begin=16, n_labels1=4, n_labels2=4,
                                    depth=3, stack_num_down=2, stack_num_up=2, patch_size=(2, 2), 
                                    num_heads=[4, 8, 8, 8], window_size=[4, 2, 2, 2], num_mlp=128, 
                                output_activation='Softmax', shift_window=True, name=model_alias)
model.summary()

In [ ]:
save_model_name = model_alias + "_DataAug_DS_" + Dataset +"_IS" + str(Image_Size) +"_EPS" + str(no_epoch) +"_OPS" + optimizers +"_LS" + losses +"_BS" + str(batch_sizes) +"_LR" + str(learning_rate).replace(".", '')
callbacks_list = generate_callbacks(model, "min", save_model_name,"val_loss" , 25, np.array([TestX[0]]), np.array([TestY[0]]) )
loss = generate_loss(losses)
optimizer = generate_optimizer(optimizers, learning_rate)
print(save_model_name)
model.compile(loss=loss, optimizer= optimizer, metrics=['accuracy'])
# model = keras.models.load_model('/home/subin/avSegmentation/1_Attention_WNet_normal_Split08_DRIVE_AV_512_500_Adam_focal_loss_4_00001', custom_objects={'focal_weighted':                   
# focal_weighted})
# print(model.optimizer.lr)
# model.optimizer.lr  = 1e-6
# print(model.optimizer.lr)

model.fit(TrainX, TrainY, validation_data=(TestX, TestY), batch_size=batch_sizes, epochs=no_epoch, verbose=1, shuffle=True, callbacks= callbacks_list)
# model.fit_generator(generator=train_generator, epochs=10, steps_per_epoch=len(train_generator), validation_data=test_generator)
# model.fit(x=train_generator, epochs=10)


1_Attention_WNet_normal_DataAug_DS_DRIVE_AV_IS512_EPS200_OPSAdam_LSfocal_loss_BS6_LR001
Epoch 1/200
27/27 [==============================] - ETA: 0s - loss: 0.0875 - accuracy: 0.7599
Epoch 1: val_loss improved from inf to 1.20377, saving model to 1_Attention_WNet_normal_DataAug_DS_DRIVE_AV_IS512_EPS200_OPSAdam_LSfocal_loss_BS6_LR001
27/27 [==============================] - 92s 2s/step - loss: 0.0875 - accuracy: 0.7599 - val_loss: 1.2038 - val_accuracy: 0.9080
Epoch 2/200


In [ ]:
# swin_wnet = keras.models.load_model('/home/subin/avSegmentation/1_Attention_WNet_normal_Split08_DRIVE_AV_500_Adam_focal_loss_4_00001', custom_objects={'focal_weighted':                   
# focal_weighted})

# # swin_wnet_old = keras.models.load_model('/home/subin/avSegmentation/Data_Aug_HRF1024_Green_Adam_1e2_focal_bs4_ep500', custom_objects={'focal_weighted':                   
# # focal_weighted})# 

In [ ]:
# swin_wnet_predicted_Test = swin_wnet.predict(TestX)

In [ ]:
# swin_wnet_predicted_Test.shape

In [ ]:
# swin_wnet_predicted_Train = swin_wnet.predict(TrainX)
# swin_wnet_predicted_Test = swin_wnet.predict(TestX)

import numpy as np
from skimage.morphology import skeletonize, erosion
from sklearn.metrics import f1_score, accuracy_score

def evaluation_code(prediction, groundtruth):
    
    encoded_pred = np.zeros(prediction.shape[:2], dtype=int)
    encoded_gt = np.zeros(groundtruth.shape[:2], dtype=int)
    
    # convert white pixels to green pixels (which are ignored)
    white_ind = np.where(np.logical_and(groundtruth[:,:,0] == 255, groundtruth[:,:,1] == 255, groundtruth[:,:,2] == 255))
    if white_ind[0].size != 0:
        groundtruth[white_ind] = [0,255,0]
        
    # translate the images to arrays suited for sklearn metrics
    arteriole = np.where(np.logical_and(groundtruth[:,:,0] == 255, groundtruth[:,:,1] == 0)); encoded_gt[arteriole] = 1
    venule = np.where(np.logical_and(groundtruth[:,:,2] == 255, groundtruth[:,:,1] == 0)); encoded_gt[venule] = 2
    arteriole = np.where(prediction[:,:,0] == 255); encoded_pred[arteriole] = 1
    venule = np.where(prediction[:,:,2] == 255); encoded_pred[venule] = 2
   
    # retrieve the indices for the centerline pixels present in the prediction
    center = np.where(np.logical_and(
        np.logical_or((skeletonize(groundtruth[:,:,0] > 0)),(skeletonize(groundtruth[:,:,2] > 0))),
        encoded_pred[:,:] > 0))
    
    encoded_pred_center = encoded_pred[center]
    encoded_gt_center = encoded_gt[center]
    
    # retrieve the indices for the centerline pixels present in the groundtruth
    center_comp = np.where(
        np.logical_or(skeletonize(groundtruth[:,:,0] > 0),skeletonize(groundtruth[:,:,2] > 0)))
    
    encoded_pred_center_comp = encoded_pred[center_comp]
    encoded_gt_center_comp = encoded_gt[center_comp]
    
    # retrieve the indices for discovered centerline pixels - limited to vessels wider than two pixels (for DRIVE)
    center_eroded = np.where(np.logical_and(
        np.logical_or(skeletonize(erosion(groundtruth[:,:,0] > 0)),skeletonize(erosion(groundtruth[:,:,2] > 0))),
        encoded_pred[:,:] > 0))
                             
    encoded_pred_center_eroded = encoded_pred[center_eroded]
    encoded_gt_center_eroded = encoded_gt[center_eroded]
    
    # metrics over full image
    cur1_acc = accuracy_score(encoded_gt.flatten(),encoded_pred.flatten())
    cur1_F1 = f1_score(encoded_gt.flatten(),encoded_pred.flatten(),average='weighted')
    print('Full image')
    print('Accuracy: {}\nF1: {}\n'.format(cur1_acc, cur1_F1))
    metrics1 = [cur1_acc, cur1_F1]
    
    # metrics over discovered centerline pixels
    cur2_acc = accuracy_score(encoded_gt_center.flatten(),encoded_pred_center.flatten())
    cur2_F1 = f1_score(encoded_gt_center.flatten(),encoded_pred_center.flatten(),average='weighted')
    print('Discovered centerline pixels')
    print('Accuracy: {}\nF1: {}\n'.format(cur2_acc, cur2_F1))
    metrics2 = [cur2_acc, cur2_F1]
    
    # metrics over discovered centerline pixels - limited to vessels wider than two pixels
    cur3_acc = accuracy_score(encoded_gt_center_eroded.flatten(),encoded_pred_center_eroded.flatten())
    cur3_F1 = f1_score(encoded_gt_center_eroded.flatten(),encoded_pred_center_eroded.flatten(),average='weighted')
    print('Discovered centerline pixels of vessels wider than two pixels')
    print('Accuracy: {}\nF1: {}\n'.format(cur3_acc, cur3_F1))
    metrics3 = [cur3_acc, cur3_F1]
    
    # metrics over all centerline pixels in ground truth
    cur4_acc = accuracy_score(encoded_gt_center_comp.flatten(),encoded_pred_center_comp.flatten())
    cur4_F1 = f1_score(encoded_gt_center_comp.flatten(),encoded_pred_center_comp.flatten(),average='weighted')
    print('Centerline pixels')
    print('Accuracy: {}\nF1: {}\n'.format(cur4_acc, cur4_F1))
    metrics4 = [cur4_acc, cur4_F1]
    
    # finally, compute vessel detection rate
    vessel_ind = np.where(encoded_gt>0)
    vessel_gt = encoded_gt[vessel_ind]
    vessel_pred = encoded_pred[vessel_ind]
    
    detection_rate = [accuracy_score(vessel_gt.flatten(),vessel_pred.flatten()),0]
    print('Amount of vessels detected: ' + str(detection_rate))
    
    return [metrics1,metrics2,metrics3,metrics4,detection_rate]
    

In [ ]:
# def create_mask(pred_mask):
#     new_label = np.zeros((Image_Size,Image_Size,3))
#     for i in range(Image_Size):
#         for j in range(Image_Size):
#             index = np.argmax(pred_mask[i][j])
#             if(index>0):
#                 new_label[i][j][index-1]=255
#     return new_label

In [ ]:
# for i in range(len(swin_wnet_predicted_Test)):
#     plt.imshow(create_mask(swin_wnet_predicted_Test[i,:,:,:]))
#     plt.show()
#     plt.imshow(TestY[i,:,:,1:4])
#     plt.show()
# #     print("Image No:",i,": ",evaluation_code(swin_wnet_predicted_Test[i,:,:,1:4], swin_wnet_predicted_Test_Label[i,:,:,1:4]))